In this project, I am going to use spacy for entity recognition on 200 Resume 
and experiment around various NLP tools for text analysis. 

I have also added skills match feature so that hiring managers can follow a metric that will
help them to decide whether they should move to the interview stage or not. 

I will be using two datasets; the first contains resume texts 
and the second contains skills that we will use to create an entity ruler.

## Inside the CSV
* ID: Unique identifier and file name for the respective pdf.
* Resume_str : Contains the resume text only in string format.
* Resume_html : Contains the resume data in html format as present while web scrapping.
* Category : Category of the job the resume was used to apply.

## Present categories
HR, Designer, Information-Technology, Teacher, Advocate, Business-Development, Healthcare, Fitness, Agriculture, BPO, Sales, Consultant, Digital-Media, Automobile, Chef, Finance, Apparel, Engineering, Accountant, Construction, Public-Relations, Banking, Arts, Aviation